# Transfer Learning

This notebook shows how to apply transfer learning on image datasets

### Import all the needed modules

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import tensorflow_backend as Ktf
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.activations import relu, sigmoid
from keras.optimizers import SGD
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.losses import binary_crossentropy
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

Using TensorFlow backend.


### Define paths for the data location and the place to save the model

Unfortunately the data used here is a private data. However it can be replaced by any healthy/unhealthy classification problem as long as the folder structure is followed.

*   training
    *   healthy
    *   unhealthy    
*   test    
    *   healthy
    *   unhealthy

In [5]:
TRAIN_DIR = '~/datasets/plant/train/'
TEST_DIR = '~/datasets/plant/test'
CKPT_DIR='vgg_16_ckpts_{epoch:03d}.hdf5'
BEST_DIR='vgg_16_best.hdf5'

### Load the images and add augmentations

The training/validation set has multiple random data transformations while the training set is just scaled for the sake of normalization

In [6]:
train_idg = ImageDataGenerator(rotation_range=360,
                               zoom_range=0.5,
                               fill_mode='reflect',
                               horizontal_flip=True,
                               vertical_flip=True,
                               validation_split=0.2,
                               rescale=1.0/255)
test_idg = ImageDataGenerator(rescale=1.0/255)

class_mode = 'binary'
classes = {
    'healthy': 0,
    'unhealthy': 1,
}

train_gen = train_idg.flow_from_directory(TRAIN_DIR, (224, 224),
                                          seed=0,
                                          batch_size=128,
                                          subset='training',
                                          class_mode=class_mode,
                                          classes=classes)

valid_gen = train_idg.flow_from_directory(TRAIN_DIR, (224, 224),
                                          seed=0,
                                          batch_size=128,
                                          subset='validation',
                                          class_mode=class_mode,    
                                          classes=classes)

test_gen = test_idg.flow_from_directory(TEST_DIR, (224, 224),
                                        class_mode=class_mode,
                                        classes=classes,
                                        shuffle=False,
                                        batch_size=1)

Found 369 images belonging to 2 classes.
Found 91 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


### Create a base model and add additional layers

Load the keras VGG model without the last/top layers and add the needed layers to solve the classification problem. Also some of the initial layers of the VGG model are set to be not trainable since only fine tuning is done to the model

In [ ]:
img_input = Input((224, 224, 3))
base_model = VGG16(include_top=False,
                   weights='imagenet',
                   input_tensor=img_input)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation=relu)(x)
predictions = Dense(1, activation=sigmoid)(x)

model = Model(inputs=img_input, outputs=predictions)

for layer in base_model.layers[:-4]:
    layer.trainable = False
    
model.compile(optimizer=SGD(0.01, decay=1e-9), loss=binary_crossentropy)

### Train the Keras model

Different callbacks are also added to improve model performance

In [ ]:
es = EarlyStopping(patience=20)
mc = ModelCheckpoint(CKPT_DIR)
bm = ModelCheckpoint(BEST_DIR, save_best_only=True)
hst = model.fit_generator(train_gen,
                          steps_per_epoch=20,
                          epochs=50,
                          callbacks=[es, mc, bm],
                          validation_data=valid_gen,
                          validation_steps=20,
                          max_queue_size=40,
                          use_multiprocessing=True)

Epoch 1/10
100/100 [==============================] - 2792s 28s/step - loss: 0.4842 - val_loss: 0.4015
Epoch 2/10
  9/100 [=>............................] - ETA: 27:25 - loss: 0.4536

### Load the training data from the python generators

Since the keras utilities create generators, they should be called to create the data

In [7]:
test_data = [next(test_gen) for _ in range(test_gen.n)]
test_data = list(zip(*test_data))
test_X, test_y = test_data
test_X = np.concatenate(test_X, axis=0)
test_y = np.concatenate(test_y, axis=0)

### Predict the result using the best model

In [8]:
best_model = load_model(BEST_DIR)
test_y_pred = best_model.predict(test_X)